<a href="https://colab.research.google.com/github/visiont3lab/deep-learning-course/blob/main/colab/TrainingPytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports



```bash
pip install torch scikit-learn plotly pandas torchsummary
```



In [1]:
# Useful
import torch
from sklearn.model_selection import train_test_split
import numpy as np
import plotly.graph_objects as go

import pandas as pd
from sklearn.metrics import confusion_matrix
import copy

# Extra
from torchsummary import summary
#!pip install torchsummary

## Load Data

In [2]:
# Dati Numpy
len_data = 500
X = np.linspace(-2,8,len_data)
Y = np.exp(0.2*X)*np.sin(3*X) - 10*np.cos(X)

# Dati Pytorch Tensor
Xt = torch.from_numpy(X).type(torch.float32).reshape(-1,1) #.unsqueeze(1)
Yt = torch.from_numpy(Y).type(torch.float32).unsqueeze(1)
print(f"X Tensor data shape: ", Xt.shape)
print(f"Y Tensor data shape: ", Yt.shape)

# Training and Test Set
X_train, X_test, Y_train, Y_test = train_test_split(Xt,Yt,test_size=0.3,shuffle=True,random_state=4)
print(f"X Train shape: {X_train.shape} , X Test shape: {X_test.shape}")

X Tensor data shape:  torch.Size([500, 1])
Y Tensor data shape:  torch.Size([500, 1])
X Train shape: torch.Size([350, 1]) , X Test shape: torch.Size([150, 1])


In [3]:
# Visualization
fig = go.Figure()
#fig.add_traces( go.Scatter(x=X, y=Y,hovertemplate='x: %{x} <br>y: %{y}',mode="markers", name="Real data") )
fig.add_traces( go.Scatter(x=X_train.flatten(), y=Y_train.flatten(), hovertemplate='x: %{x} <br>y: %{y}',mode="markers", name="Train data") )
fig.add_traces( go.Scatter(x=X_test.flatten(), y=Y_test.flatten(), hovertemplate='x: %{x} <br>y: %{y}',mode="markers", name="Test data") )

fig.update_layout(title="Funzione di stimare")
fig.show()

In [4]:
# Tensor Dataset Che converte i dati da numpy a Pytorch
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, x,y):
        self.x = x
        self.y = y
    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]
        return x, y
    def __len__(self):
        return self.x.shape[0]

# Dataset generator creation
train_ds = MyDataset(X_train,Y_train)
test_ds = MyDataset(X_test,Y_test)

In [8]:
# Get data using iterator list
for x,y in train_ds:
  print(f"x value: {x} , x shape {x.shape}, y value: {y}, y shape: {y.shape}")
  break

x value: tensor([4.6533]) , x shape torch.Size([1]), y value: tensor([3.0869]), y shape: torch.Size([1])


In [109]:
# Get value using iterator
(x,y) = next(iter(train_ds))
print(f"x value: {x} , x shape {x.shape}, y value: {y}, y shape: {y.shape}")

x value: tensor([4.6533]) , x shape torch.Size([1]), y value: tensor([3.0869]), y shape: torch.Size([1])


In [110]:
# Data loader
train_dl = torch.utils.data.DataLoader(train_ds,batch_size=10,shuffle=True)
test_dl = torch.utils.data.DataLoader(test_ds,batch_size=5,shuffle=True)

# Get data using iterator list
for x,y in train_dl:
  print(f"Train: x shape {x.shape}, y shape: {y.shape}")
  break
  
for x,y in test_dl:
  print(f"Test: x shape {x.shape}, y shape: {y.shape}")
  break

Train: x shape torch.Size([10, 1]), y shape: torch.Size([10, 1])
Test: x shape torch.Size([5, 1]), y shape: torch.Size([5, 1])


## Neural Network

In [6]:
class MyNet(torch.nn.Module):
    def __init__(self):
        super(MyNet,self).__init__()
        self.fc1 = torch.nn.Linear(1,100)
        self.fc2 = torch.nn.Linear(100,50)
        self.fc3 = torch.nn.Linear(50,1)
        
        #print(self.fc2.weight, self.fc2.bias)
    def forward(self,x):
        # torch.sigmoid, torch.tanh, torch.relu
        z = torch.tanh(self.fc1(x)) 
        c = torch.tanh(self.fc2(z)) 
        y = self.fc3(c)
        return y

t = torch.tensor([
    [4],
    [4],
    [3],
    [43],
    [4],
], dtype=torch.float32)
print(t.shape)

net = MyNet()

y = net(t)
print(y)
#summary(net, (1,1), batch_size=-1, device='cpu')

torch.Size([5, 1])
tensor([[0.6730],
        [0.6730],
        [0.6174],
        [0.7885],
        [0.6730]], grad_fn=<AddmmBackward0>)


In [142]:
# Get model weight and biases
for name, param in net.named_parameters():
  #print(f"name: {name}, param: {param}")
  
  # Freeze layers
  # param.requires_grad = False
  #if "fc3" not in name:
  #  param.requires_grad = False

  print(f"name: {name}, param gradient: {param.requires_grad}")

name: fc1.weight, param gradient: True
name: fc1.bias, param gradient: True
name: fc2.weight, param gradient: True
name: fc2.bias, param gradient: True
name: fc3.weight, param gradient: True
name: fc3.bias, param gradient: True


In [143]:
# Update a weight or bias with custom initial value
print(f"fc1.weight shape: {net.fc1.weight.shape}, fc1.bias shape: {net.fc1.bias.shape}")
v = 2*torch.ones(100,1).type(torch.float32)
net.fc1.weight = torch.nn.Parameter(v)
#net.fc1.weight

fc1.weight shape: torch.Size([100, 1]), fc1.bias shape: torch.Size([100])


## Training Loop


In [93]:
def my_metric(target, output):
    # Comptue mean squaer error (Migliora quanto piu' ci avviciniamo a zero)
    mse = torch.sum( torch.abs(output - target) )
    return mse

# Params
device = torch.device("cuda:0")
#device = torch.device("cpu")
net = MyNet()
net = net.to(device)
loss_func = torch.nn.MSELoss(reduction="sum")  
metric_func = my_metric
opt = torch.optim.Adam(net.parameters(),lr=0.001)
num_epochs = 20
path2weigths = "best_model.pt"
train_dl = torch.utils.data.DataLoader(train_ds,batch_size=3,shuffle=True)
test_dl = torch.utils.data.DataLoader(test_ds,batch_size=10,shuffle=True)

In [98]:
# one epoch training
def loss_epoch(net, loss_func, metric_func, dataset_dl, opt , device):
    
    # -- init
    loss = 0.0
    metric = 0.0
    len_data = float(len(dataset_dl.dataset))
   
    # -- Iterate over the  batch data
    for xb,yb in dataset_dl:    
        
        # -- send to cuda the data (batch size)
        xb = xb.to(device)
        yb = yb.to(device)
        
        # -- obtain model output 
        yb_h = net.forward(xb)
    
        # -- loss and metric Calculation
        loss_b = loss_func(yb_h, yb)

        # -- obtain performance metric 
        with torch.no_grad():
          metric_b = metric_func(yb,yb_h)

        # -- Update step
        if opt is not None:
          # Compute derivative
          loss_b.backward()
          # Update weights
          opt.step()
          # zero the parameter gradients
          opt.zero_grad()
    
        # Batch loss and metric update 
        loss += loss_b.item()
        metric += metric_b.item() 

    # average loss
    loss = loss / len_data
    # average metric
    metric = metric / len_data
  
    return loss, metric

#train_loss, train_metric = loss_epoch( net, loss_func, metric_func, train_dl, opt, device )
#test_loss, test_metric = loss_epoch( net, loss_func, metric_func, test_dl, opt, device )

In [99]:
def train(num_epochs, train_dl, test_dl, loss_func, metric_func, opt, device, path2weigths):

  # --> Metrics to plot data
  loss_history={"train": [],"test":[]}
  metric_history={"train": [],"test":[]}
  best_loss=float("inf")

  # -- main loop
  for epoch in range(num_epochs):
    
    # -- Log
    print("----\nEpoch %s/%s" % (epoch+1,num_epochs))

    # --> Set the model to train stage
    net.train()

    # -- One epoch train step
    train_loss, train_metric = loss_epoch(net, loss_func, metric_func, train_dl, opt, device)

    # -- Collect loss and metric for training dataset
    loss_history["train"].append(train_loss)
    metric_history["train"].append(train_metric)

    # -- Set the model  (validation) mode
    net.eval()

    # One epoch validation step
    with torch.no_grad():
      test_loss, test_metric = loss_epoch(net, loss_func, metric_func, test_dl, opt=None, device=device)

      # --> collect loss and metric for test dataset
      loss_history["test"].append(test_loss)
      metric_history["test"].append(test_metric)

      # --> store best model
      if test_loss < best_loss:
        print("--> model improved! --> saved to %s" %(path2weigths))
        best_loss = test_loss
        # save weights
        best_model_weights = copy.deepcopy(net.state_dict())
        # store weights 
        torch.save(net.state_dict(), path2weigths)
    
    #net.load_state_dict(best_model_weights)
    print("--> train_loss: %.6f, test_loss: %.6f, train_metric: %.3f, test_metric: %.3f" % (train_loss,test_loss,train_metric,test_metric))

  return loss_history, metric_history

loss_history, metric_history = train(num_epochs, train_dl, test_dl, loss_func, metric_func, opt, device, path2weigths)

----
Epoch 1/20
--> model improved! --> saved to best_model.pt
--> train_loss: 1.970062, test_loss: 1.414703, train_metric: 0.933, test_metric: 0.718
----
Epoch 2/20
--> train_loss: 1.844233, test_loss: 1.838555, train_metric: 0.902, test_metric: 0.970
----
Epoch 3/20
--> train_loss: 1.829686, test_loss: 1.430910, train_metric: 0.910, test_metric: 0.826
----
Epoch 4/20
--> model improved! --> saved to best_model.pt
--> train_loss: 1.554401, test_loss: 1.019830, train_metric: 0.813, test_metric: 0.599
----
Epoch 5/20
--> train_loss: 1.381985, test_loss: 1.022714, train_metric: 0.790, test_metric: 0.636
----
Epoch 6/20
--> model improved! --> saved to best_model.pt
--> train_loss: 1.275342, test_loss: 0.918174, train_metric: 0.750, test_metric: 0.585
----
Epoch 7/20
--> model improved! --> saved to best_model.pt
--> train_loss: 1.156214, test_loss: 0.853183, train_metric: 0.730, test_metric: 0.598
----
Epoch 8/20
--> train_loss: 1.064924, test_loss: 0.868503, train_metric: 0.714, test_me

## Plot results

In [100]:
fig_loss = go.Figure()
fig_metric = go.Figure()

x = [i+1 for i in range(num_epochs)]

fig_loss.add_traces( go.Scatter(x=x,y=loss_history["train"], name="train loss", mode="lines+markers" ) )
fig_loss.add_traces( go.Scatter(x=x,y=loss_history["test"], name="test loss"  , mode="lines+markers") )
fig_loss.update_layout(title="Loss Results", xaxis_title="epochs", hovermode="x")
fig_loss.show()

fig_metric.add_traces( go.Scatter(x=x,y=metric_history["train"], name="train metric", mode="lines+markers") )
fig_metric.add_traces( go.Scatter(x=x,y=metric_history["test"], name="test_metric" , mode="lines+markers") )
fig_metric.update_layout(title="Metric Results", xaxis_title="epochs", hovermode="x")
fig_metric.show()

## Test

In [101]:
# Run on cpu
device = torch.device("cpu")

# Load Regression
model = MyNet()
weights = torch.load("best_model.pt")
model.load_state_dict(weights)
model = model.to(device)

# Predict Regression
Xt = torch.from_numpy(X).type(torch.float32).unsqueeze(1)
Y_hat = model.forward(Xt).detach().numpy().reshape(-1)

# Visualize Regression
fig = go.Figure()
fig.add_traces( go.Scatter(x=X, y=Y, name="Real",hovertemplate='x: %{x} <br>y: %{y}') )
fig.add_traces( go.Scatter(x=X, y=Y_hat, name="Predicted",hovertemplate='x: %{x} <br>y: %{y}') )
fig.update_layout(title="Regression Results",hovermode="x")
fig.show()